<a href="https://colab.research.google.com/github/ywchanna2001/Big-Data-Processing/blob/main/Spark_ML_Lib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark